In [1]:
import sys
import os
copypath = ['../modules', 
            '../Train', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python27.zip', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/plat-linux2', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-tk', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-old', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-dynload', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/site-packages', 
            '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg']
for p in reversed(copypath):
    sys.path.insert(0, p)
print sys.path
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['DEEPJET'] = '../../DeepJet'
#os.environ['LD_LIBRARY_PATH'] = '../modules:%s'%os.environ['LD_LIBRARY_PATH']
#os.environ['PATH'] = '../scripts:%s'%os.environ['PATH']
#os.environ['CONDA_PREFIX'] = '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu'
#os.environ['LD_PRELOAD'] = '%s/lib/libmkl_core.so:%s/lib/libmkl_sequential.so'%(os.environ['CONDA_PREFIX'],os.environ['CONDA_PREFIX'])
import keras
keras.backend.set_image_data_format('channels_last')

['../modules', '../Train', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python27.zip', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/plat-linux2', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-tk', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-old', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/lib-dynload', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/site-packages', '/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '', '/opt/conda/envs/python2/lib/python27.zip', '/opt/conda/envs/python2/lib/python2.7', '/opt/conda/envs/python2/lib/python2.7/plat-linux2', '/opt/conda/envs/python2/lib/python2.7/li

Using TensorFlow backend.


In [2]:
class MyClass:
    """A simple example class"""
    def __init__(self):
        self.inputDataCollection = ''
        self.outputDir = ''

In [5]:
!rm -r train_deepsimple_nobatchnorm_1e-6
!nvidia-smi

rm: cannot remove ‘train_deepsimple_nobatchnorm_1e-6’: No such file or directory
Mon Sep 18 04:04:16 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.66                 Driver Version: 375.66                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 0000:04:00.0     Off |                  N/A |
| 27%   35C    P8     9W / 180W |      0MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 0000:05:00.0     Off |                  N/A |
| 27%   34C    P8     9W / 180W |      0MiB /  8114MiB |      0%      Default |
+------

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

from training_base import training_base
from Losses import loss_NLL
import sys

args = MyClass()
args.inputDataCollection = '/inputdata/BumbleB/convert_deepDoubleB_simple_train_val/dataCollection.dc'
args.outputDir = 'train_deepsimple_nobatchnorm_1e-6'

#also does all the parsing
train=training_base(testrun=False,args=args)


if not train.modelSet():
    from DeepJet_models_ResNet import deep_model_doubleb_nobatchnorm

    train.setModel(deep_model_doubleb_nobatchnorm)
    
    train.compileModel(learningrate=0.000001,
                       loss=['categorical_crossentropy'],
                       metrics=['accuracy'])
    

model,history,callbacks = train.trainModel(nepochs=500, 
                                 batchsize=8192, 
                                 stop_patience=1000, 
                                 lr_factor=0.5, 
                                 lr_patience=10, 
                                 lr_epsilon=0.0000001, 
                                 lr_cooldown=2, 
                                 lr_minimum=0.0000001, 
                                 maxqsize=100)

[[1, 27]]
(?, 1, 27)
(?, ?)
(?, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 27)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 27)                0         
_________________________________________________________________
fc1_relu (Dense)             (None, 16)                448       
_________________________________________________________________
fc1_dropout (Dropout)        (None, 16)                0         
_________________________________________________________________
softmax (Dense)              (None, 2)                 34        
Total params: 482.0
Trainable params: 482.0
Non-trainable params: 0.0
_________________________________________________________________
None
Epoch 1/500
1947/1952 [============================>.] - ETA: 0s - loss: 1.8795 - acc: 0.1514
***c

1951/1952 [============================>.] - ETA: 0s - loss: 0.6718 - acc: 0.9732
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00011: val_loss improved from 0.67301 to 0.67118, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00011: saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_model_last.h5

***callbacks end***

1952/1952 [==============================] - 14s - loss: 0.6718 - acc: 0.9732 - val_loss: 0.6712 - val_acc: 0.9667
Epoch 13/500
1944/1952 [============================>.] - ETA: 0s - loss: 0.6700 - acc: 0.9735
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00012: val_loss improved from 0.67118 to 0.66947, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00012: s

1952/1952 [==============================] - 15s - loss: 0.6505 - acc: 0.9731 - val_loss: 0.6501 - val_acc: 0.9672
Epoch 25/500
1941/1952 [============================>.] - ETA: 0s - loss: 0.6487 - acc: 0.9737
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00024: val_loss improved from 0.65015 to 0.64831, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00024: saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_model_last.h5

***callbacks end***

1952/1952 [==============================] - 15s - loss: 0.6486 - acc: 0.9738 - val_loss: 0.6483 - val_acc: 0.9683
Epoch 26/500
1946/1952 [============================>.] - ETA: 0s - loss: 0.6469 - acc: 0.9733
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00025: val_loss improved from 0.64831 to 0.64

1952/1952 [==============================] - 15s - loss: 0.6279 - acc: 0.9737 - val_loss: 0.6280 - val_acc: 0.9671
Epoch 38/500
1948/1952 [============================>.] - ETA: 0s - loss: 0.6262 - acc: 0.97371750/1952 [=========================>....] - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.6262 - acc: 0.97
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00037: val_loss improved from 0.62800 to 0.62627, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00037: saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_model_last.h5

***callbacks end***

1952/1952 [==============================] - 15s - loss: 0.6262 - acc: 0.9737 - val_loss: 0.6263 - val_acc: 0.9675
Epoch 39/500
1944/1952 [============================>.] - ETA: 0s - loss: 0.6245 - acc: 0.9734- ETA: 2s - loss: 0.6248 - acc: 0.97 - E
***callbacks***
saving

1952/1952 [==============================] - 19s - loss: 0.6061 - acc: 0.9734 - val_loss: 0.6064 - val_acc: 0.9675
Epoch 51/500
1948/1952 [============================>.] - ETA: 0s - loss: 0.6045 - acc: 0.9730
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00050: val_loss improved from 0.60642 to 0.60488, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00050: saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_model_last.h5

***callbacks end***

1952/1952 [==============================] - 17s - loss: 0.6045 - acc: 0.9731 - val_loss: 0.6049 - val_acc: 0.9671
Epoch 52/500
1947/1952 [============================>.] - ETA: 0s - loss: 0.6028 - acc: 0.9734
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00051: val_loss improved from 0.60488 to 0.60

1952/1952 [==============================] - 20s - loss: 0.5849 - acc: 0.9733 - val_loss: 0.5855 - val_acc: 0.9677
Epoch 64/500
1946/1952 [============================>.] - ETA: 0s - loss: 0.5833 - acc: 0.9731
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00063: val_loss improved from 0.58549 to 0.58386, saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_best_model.h5
Epoch 00063: saving model to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6//KERAS_check_model_last.h5

***callbacks end***

1952/1952 [==============================] - 19s - loss: 0.5833 - acc: 0.9732 - val_loss: 0.5839 - val_acc: 0.9678
Epoch 65/500
1943/1952 [============================>.] - ETA: 0s - loss: 0.5817 - acc: 0.9731
***callbacks***
saving losses to /home/jovyan/work/DeepJet/Train/train_deepsimple_nobatchnorm_1e-6/losses.log
Epoch 00064: val_loss improved from 0.58386 to 0.58

Process Process-14692:
KeyboardInterrupt
Traceback (most recent call last):
  File "/opt/conda/envs/python2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/envs/python2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "../modules/TrainData.py", line 45, in _read_arrs_
    h5f[idstr].read_direct(arl[i])
  File "/inputdata/BumbleB/jduarte/miniconda2/envs/deepjetLinux3_gpu/lib/python2.7/site-packages/h5py/_hl/dataset.py", line 641, in read_direct


KeyboardInterrupt: 

    self.id.read(mspace, fspace, dest, dxpl=self._dxpl)
